<a href="https://colab.research.google.com/github/poongodimsa/DataScience-projects-for-vidiq/blob/main/Script_DS_Project1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 -  Revenue by customer by month - V 1

### Version 2 - Specs


---



#### Let's call all files "Platform YY.MM.DD", so if we ran the Stripe output today, it would be called "Stripe 22.06.17", to be applied to PayPal and ChartMogul as well. Can you also add a folder in the "Results" folder called Archives, and put all old output files in there?
--> Will rename accordingly. I will code the script to move all the files under 'Results' to 'Archive' before creating a new file for the run. 

** Stripe Output: **


In the output file:Country: In the Stripe raw data files, column AK "card_country" provides a 2 digit country code. Pls use this.Plan Name: In the Stripe raw data files, col AL "statement_descriptor" provides the Plan. Pls use this for Customer Plan in the output.Plan Duration - Appears unavailable here
Customer Name - Appears unavailable hereLet's add an index column for Customer Email. This might work in lieu of a Name. Note we'll want to do this in PayPal too, so all Indexes match. That might mean Customer Unique ID will be something else for PayPal.


PayPal Output:

Country: 
There is a 2 digit country code like Stripe, we should try to have data conventions match if possible, so pls use this where we can. If this is N/A, your current output pulls from the PayPal raw data col "Country". However, there are many N/As. Finally, country is also provided in the last part of the "Shipping Address". Pls have your script try to pull from these three in this order, so if the 2 digit code is avail, pull that. If not, pull from "country". If that's not available, from col N "Shipping Address" by slicing the string off up to the last comma.  Between the 3 of these, we should be able to get nearly all countries. Plan: Can be found in Subject. It should either be Pro, Boost or Max. Your current output has some inaccurate info for plans - for example, we don't have a "Gold Plan", so not sure that that is. Duration: This can also be found in Subject, there is says "monthly" or "yearly". Just do a string search and if either of these are found in the row, add them to the duration.


### Version 1 - Specifications - For Paypal only

####  Final result:

1. Get EUR customer email addresses to USD amounts from EUR amounts

2. Remove /ignore all EUR amounts from the calculation. If you add all EUR (net) together. This will give you zero value

3. Ignore any amounts under "payments@vid.io" - Anway there will be no amounts for this email if you do the above two steps


#### Description:

Some of our customers pay in EUR. Since our default currency is USD, Paypal immediately converts these EUR payments to USD

The Problem we have; See the below screenshot taken from the 2022.02 Payroll data file. (A:5 to A:7)

Patrick is a EU customer. he paid us in EU currency. EUR 10.00
Paypal immediately converted to USD 9.84
You can also see that when you add both EUR amounts it will net set off ( Zero value)
So in these multi-currency cases, we need to get the USD values only. But the problem; there is no email address as the unique ID assigned to USD amounts. It is blank

The workaround here is 
In the above case, the Customer's email is knopey@gmx.de. But this is assigned to the EUR amount instead of the USD amount.

Transaction ID - 5RE84019JL6683131 This is the reference for the EUR amount. But we have a different reference for the USD amount. So we cannot use that get the customer email

Reference Txn ID - 5RE84019JL6683131 - Check the reference under "Reference Txn ID" for the USD amount now compare this with the above transaction ID. You will see there is a match
In Excel, we can use the "VlookUp" formula to get the customer email to USD amount using these two different IDs
Now please find a way to get the correct email ID to USD amount

**Important: Consider 'To Mail address' as unique id for debit transactions.**

### Version 0 - Specifications
---



Headers - Months in format “YYYY-MM”

Row Indexes - 
1.	Customer Name
2.	Platform (Stripe or PayPal)
3.	Customer Unique ID
4.	Customer Country (if available)
5.	Customer Plan (Pro or Boost; if available)
6.	Customer Plan Duration (Monthly or Yearly, if available)

Rows - Unique customers

Values - sum of total payments (received and refunded) per month for each customer

1.	Stripe - 

○	use "customer_id" as the unique ID to get the data
2.	PayPal

○	Important -----> Consider Only "Subscription Payment" & "Payment Refund" types ("E" column in the PayPal statements). Remove all the other types

○	Use "From Email Address" as the unique ID to get the data ( There is no special code for customer) 

3.	Ignore ChartMogul (“CM”) for now. This will be used in the second iteration.

Output

●	should be an excel file with one tab of revenue per customer per month

●	Sort by start month


Stripe

Customer Plan - You can map to "statement_descriptor" ( AL Column)

Customer Plan Duration - N/A - You Can keep this blank

Sum of total payments per customer ------> Always use Gross Amounts (G Column)

Paypal

Customer Unique ID - map to 'From Email Address? - "YES"

Customer Plan - You Can map the "Subject" /  "AD" Column 

Customer Plan Duration - N/A - You Can keep this blank

Amounts -----> Always use Gross Amounts / "H" Column



### Script
---

In [1]:
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import *


Mount Gdrive

In [2]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Data Science 2022/"

Mounted at /content/gdrive


In [3]:
def get_month_headers(startmonth,monthcount):
  '''
  Function: Returns list of dates starting from 'startmonth' incrementing till 'monthcount' counts
  startdate: date in format 'YYYY-MM' as string datatype
  monthcount: Number of increments
  '''

  lstout = [] #Intialize

  start = datetime.strptime(startmonth,'%Y-%m') 
  for c in range(monthcount):
    lstout.append((start + relativedelta(months=+c)).strftime('%Y-%m'))
    
  return(lstout)

def gen_header():
  lstheader = [ "Customer Name","Platform","Customer Unique ID", "Customer Country", "Customer Plan", "Customer Plan Duration" ]
  lstmonths = get_month_headers('2020-01', 27)
  return(lstheader + lstmonths)


#### Stripe

Loop through the input files to calculate revenue per customer

In [ ]:
#Set file paths
inpath = path + "Stripe/"
outpath = path + "Results/"
archpath = path + "Results/Archives"

if not os.path.isdir(archpath):
    os.mkdir(archpath)

c=0
for file in os.listdir(inpath):
  c+=1
  print(file)
  df = pd.read_csv(inpath + file, index_col=False)

  #Change the data type of gross to numeric
  df = df.astype({"gross": str})
  df['gross'] = df['gross'].str.replace(',','')
  df = df.astype({"gross": float})


  #*********** Calculate revenue per customer per month ******************
  #Apply Group by to input data
  sum_df = df.groupby(['customer_id' ], as_index=False).agg({'customer_name':'first', 'customer_id':'first',
                                                            'shipping_address_country':'first', 'statement_descriptor':'first','gross': 'sum'}) #grouped by dataframe'
  sum_df = sum_df.sort_values('customer_id')
  
  #Form revenue headers
  month = '20' + file[7:12].replace('.','-') #From input file name
  sum_df.rename(columns={'customer_name':'Customer Name', 'customer_id':'Customer Unique ID', 
                        'shipping_address_country': "Customer Country",'statement_descriptor':'Customer Plan', 'gross': month},inplace = True)
  
  
  #Create base dataframe from first file
  if c==1:
    final_df=sum_df.copy()
  else:
 
    #Merge using full outer join
    final_df = final_df.merge(sum_df, how='outer', on = 'Customer Unique ID')

    #Drop columns with '_x
    cols = [col for col in final_df.columns if col.lower()[-2:] != '_x']
    final_df=final_df[cols]

    #Remove '_y' from column name
    for col in final_df.columns:
      if col.lower()[-2:] == '_y':
        final_df.rename(columns = {col: col[:-2]}, inplace=True)

#insert 'Platform' and 'customer plan duration' cols
final_df.insert(1,"Platform", "Stripe")
final_df.insert(1,'Customer Plan Duration',"")


#Reorder the headers
lsthead = gen_header()
final_df = final_df.reindex(columns=lsthead)

#Sort by first month
final_df = final_df.sort_values('2020-01')

#Export data to excel without index
final_df.to_excel(outpath + 'Stripe_RevenueByCustomerByMonth.xlsx',index = False)

print('done')

stripe_20.01.csv
stripe_20.02.csv
stripe_20.03.csv
stripe_20.04.csv
stripe_20.05.csv
stripe_20.06.csv
stripe_20.07.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (19,20,38,42,43,45,47,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


stripe_20.08.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,43,47,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


stripe_20.09.csv
stripe_20.10.csv


...

stripe_21.11.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


stripe_21.12.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (44,47,50,52,57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


stripe_22.03.csv
stripe_22.02.csv
stripe_22.01.csv
done


#### PayPal

V1 - Requirements
1. 

In [ ]:
#Set file paths
inpathpp = path + "PayPal/"
outpath = path + "Results/"

c=0
for file in os.listdir(inpathpp):
  c+=1
  print(file)
  
  #Read file to dataframe
  df = pd.read_csv(inpathpp + file, encoding = "ISO-8859-1", index_col=False)

  #Consider Only "Subscription Payment" & "Payment Refund" types ("E" column in the PayPal statements). Remove all the other types
  #Include "General Currency Conversion" in the Filter to get USD for other currencies
  df = df[(df["Type"] == "Subscription Payment") | (df["Type"] == "Payment Refund") | (df["Type"] == "General Currency Conversion")].copy()

  df = df[["From Email Address","To Email Address","Name","Country","Subject", "Gross","Balance Impact",
          "Currency","Type","Transaction ID","Reference Txn ID"]].copy()

  #Unique Id - If 'Balance Impact" is 'Credit' use 'From email address' else use 'To Email Address'
  dftemp = pd.DataFrame(columns=['Email'])
  for ind,row in df.iterrows():
    if row['Balance Impact'] == "Credit":
      dftemp.loc[len(dftemp.index)] = [row["From Email Address"]]
    elif row['Balance Impact'] == "Debit":
      dftemp.loc[len(dftemp.index)] = [row["To Email Address"]]
    else:
      dftemp.loc[len(dftemp.index)] = ""

  #Reset index to avoid null values
  dftemp.reset_index(drop=True, inplace = True)
  df.reset_index(drop=True, inplace = True)

  #Add email column to the base df
  df = pd.concat([df,dftemp], axis = 1)

  #************************************* Logic to get email from other currency record to USD ******************

  #DF to copy email from 
  dffrom = df[ (df['Currency'] != "USD") & (( df['Type'] == "Subscription Payment") | (df["Type"] == "Payment Refund")) ].copy()

  # df copy to
  dfto = df[ (df['Currency'] == "USD") & ( df['Type'] == "General Currency Conversion")].copy()

  #copy email
  for ind,row in dffrom.iterrows():
    txnid = row['Transaction ID']
    for ind1,row1 in dfto.iterrows():
      if (row1['Reference Txn ID'] == txnid):
        dfto.loc[ind1,'Email'] = row['Email']
        dfto.loc[ind1,'Name'] = row['Name']
        dfto.loc[ind1,'Country'] = row['Country']
        dfto.loc[ind1,'Subject'] = row['Subject']

  #Delete all empty 'Email' entries from dfto as it does not comply with 'Type' -'subs pay' or 'pay refund'
  dfto = dfto [(pd.isnull(dfto['Email']) == False)].copy()

  #Reset index to avoid null values
  dfto.reset_index(drop=True, inplace = True)
  df.reset_index(drop=True, inplace = True)

  dfnew = pd.concat([df,dfto], axis = 0)
  dfnew.reset_index(drop=True, inplace = True)

  #Now we have email ids for USD records
  #Delete all currencies other than 'USD' and all entries with blank 'Email'
  df = dfnew[(dfnew['Currency'] == "USD") & (pd.isnull(dfnew['Email']) == False)].copy()
  
  df = df[["Email","Name","Country","Subject", "Gross"]].copy()


  #Change the data type of gross to numeric
  df = df.astype({"Gross": str})
  df['Gross'] = df['Gross'].str.replace(',','')
  df = df.astype({"Gross": float})


  #************************* Calculate revenue per customer per month ******************

  #Apply Group by to input data
  sum_df = df.groupby(['Email' ], as_index=False).agg({'Name':'first', 'Email':'first',
                                                            'Country':'first', 'Subject':'first','Gross': 'sum'})     #grouped by dataframe'
  sum_df = sum_df.sort_values('Email')
  
  #Form revenue headers
  month =  file[7:14].replace('.','-')  #From input file name

  sum_df.rename(columns={'Name':'Customer Name', 'Email':'Customer Unique ID', 
                        'Country': "Customer Country",'Subject':'Customer Plan', 'Gross': month},inplace = True)
  
  
  #Create base dataframe from first file
  if c==1:
    final_df=sum_df.copy()
  else:
 
    #Merge using full outer join
    final_df = final_df.merge(sum_df, how='outer', on = 'Customer Unique ID')

    #Drop columns with '_x
    cols = [col for col in final_df.columns if col.lower()[-2:] != '_x']
    final_df=final_df[cols]

    #Remove '_y' from column name
    for col in final_df.columns:
      if col.lower()[-2:] == '_y':
        final_df.rename(columns = {col: col[:-2]}, inplace=True)
  
#insert 'Platform' and 'customer plan duration' cols
final_df.insert(1,"Platform", "PayPal")
final_df.insert(1,'Customer Plan Duration',"")

#Reorder the headers
lsthead = gen_header()
final_df = final_df.reindex(columns=lsthead)

#Sort by first month
final_df = final_df.sort_values('2020-01')

#Export data to excel without index
final_df.to_excel(outpath + 'PayPal_RevenueByCustomerByMonth_V1.xlsx',index = False)

print('done')  

Paypal_2020.01.csv
Paypal_2020.02.csv
Paypal_2020.03.csv
Paypal_2020.04.csv
Paypal_2020.05.csv
Paypal_2020.06.csv
Paypal_2020.07.csv
Paypal_2020.08.csv
Paypal_2020.09.csv
Paypal_2020.10.csv
Paypal_2020.11.csv
Paypal_2020.12.csv
Paypal_2021.01.csv
Paypal_2021.02.csv
Paypal_2021.03.csv


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Paypal_2021.04.CSV
Paypal_2021.05.CSV
Paypal_2021.06.csv
Paypal_2021.07.csv
Paypal_2021.08.csv
Paypal_2021.09.CSV
Paypal_2021.10.csv
Paypal_2021.11.csv
Paypal_2021.12.csv
Paypal_2022.01.csv
Paypal_2022.02.csv
Paypal_2022.03.csv
done
